In [33]:
# Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv 
# (либо его заархивированную версию salary-train.zip).

In [102]:
import pandas as pd
import numpy as np

In [83]:
data_train = pd.read_csv('_salary-train.csv')

In [84]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [85]:
data_test = pd.read_csv('_salary-test-mini.csv')

In [90]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [37]:
# Проведите предобработку

In [38]:
# Приведите тексты к нижнему регистру (text.lower())

In [86]:
data_train['FullDescription'] = data_train['FullDescription'].str.lower()
data_test['FullDescription'] = data_test['FullDescription'].str.lower()

In [41]:
# Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
# Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). 
# Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты

In [87]:
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [44]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5)
Xtrain = vectorizer.fit_transform(data['FullDescription'])

In [89]:
Xtest = vectorizer.transform(data_test['FullDescription'])

In [46]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. 
# Код для этого был приведен выше.

In [47]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

In [48]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime

In [49]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [92]:
X_test = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [50]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки". 
# Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. 
# Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [93]:
from scipy.sparse import hstack

In [94]:
data_new_train = hstack([Xtrain, X_train])

In [95]:
data_new_test = hstack([Xtest, X_test])

In [53]:
# Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. 
# Целевая переменная записана в столбце SalaryNormalized

In [97]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0, random_state = 241)
clf.fit(data_new_train, data_train['SalaryNormalized'])

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [ ]:
# Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
# Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [99]:
n = clf.predict(data_new_test)

In [100]:
n

array([56555.61500155, 37188.32442618])

In [103]:
for i in n:
    print(np.round(i,decimals = 2))

56555.62
37188.32
